<a href="https://colab.research.google.com/github/jafcn09/IA_1/blob/main/Semana_4.2_Coarse_finer_search_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVC, SVR
from sklearn.metrics import SCORERS
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
from scipy import stats
from sklearn.model_selection import  RandomizedSearchCV


# dataset

In [ ]:
X, y = datasets.load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape

(353, 10)

# pipeline

In [ ]:
#pipeline
model_pipeline = Pipeline([('scaler', StandardScaler()),
                           ('svr', SVR(C= 1, kernel = 'rbf', gamma = 1))])

# estrategia coarse con grid search

In [ ]:
grid_list = {"svr__C": [0.1, 10, 100, 1000],
             "svr__gamma": [0.01, 0.1, 1, 10]}

grid_search = GridSearchCV(model_pipeline, param_grid = grid_list, n_jobs = 4, cv = 3, scoring = 'neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svr', SVR(C=1, gamma=1))]),
             n_jobs=4,
             param_grid={'svr__C': [0.1, 10, 100, 1000],
                         'svr__gamma': [0.01, 0.1, 1, 10]},
             scoring='neg_root_mean_squared_error')

In [ ]:
best_C = grid_search.best_params_['svr__C']
best_gamma = grid_search.best_params_['svr__gamma']


In [ ]:
print("best_C", best_C)
print("best_gamma", best_gamma)

best_C 100
best_gamma 0.01


estrategia finer con random

In [ ]:
rand_list = {"svr__C": stats.uniform(best_C - 10, best_C + 10),
             "svr__gamma": stats.uniform(best_gamma - 0.005 , best_gamma + 0.5)}

rand_search = RandomizedSearchCV(model_pipeline, param_distributions = rand_list, n_iter = 300, n_jobs = 4, cv = 3, random_state = None, scoring = 'neg_root_mean_squared_error')
rand_search.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('svr', SVR(C=1, gamma=1))]),
                   n_iter=300, n_jobs=4,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d2d6ac292a0>,
                                        'svr__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7d2d6ac72740>},
                   scoring='neg_root_mean_squared_error')

# mostrar resultados en dataframe

In [ ]:
import pandas as pd
results_df = pd.DataFrame(rand_search.cv_results_).sort_values(by=["rank_test_score"])
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

,params,rank_test_score,mean_test_score,std_test_score
194,"{'svr__C': 104.11698379840243, 'svr__gamma': 0...",1,-53.779147,0.768852
51,"{'svr__C': 141.38049289793315, 'svr__gamma': 0...",2,-53.874705,1.067682
229,"{'svr__C': 134.93332870478065, 'svr__gamma': 0...",3,-53.904705,1.150436
107,"{'svr__C': 97.96471088945572, 'svr__gamma': 0....",4,-53.941371,1.123249
253,"{'svr__C': 123.53038208789394, 'svr__gamma': 0...",5,-53.951263,0.707150
...,...,...,...,...
246,"{'svr__C': 174.31044003912427, 'svr__gamma': 0...",296,-64.812194,2.369733
219,"{'svr__C': 162.76771377549807, 'svr__gamma': 0...",297,-64.834747,2.356103
125,"{'svr__C': 169.66650749198462, 'svr__gamma': 0...",298,-64.863859,2.382800
299,"{'svr__C': 148.00521190037693, 'svr__gamma': 0...",299,-64.871503,2.343941


In [ ]:
rand_search.best_params_

{'svr__C': 104.1696684910979, 'svr__gamma': 0.03566164552475197}

# validación del modelo en el conjunto de test

In [ ]:
from sklearn.metrics import mean_squared_error
# score RMSE en el conjunto de test
np.sqrt(mean_squared_error(rand_search.predict(X_test), y_test))

60.69183680164452